<a href="https://colab.research.google.com/github/HolyPrapor/InfinitiveOrFiniteClassification/blob/main/NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Введение

Эта работа посвящена решению проблемы классификации окончания "тся/ться" в глаголах.

Более конкретно, конечная задача формулируется как "Необходимо научиться корректировать ошибки в предложениях с неверным использованием 'тся/ться'".

В этой работе задействованы данные из [проекта Natasha](https://natasha.github.io/).

Нам пригодятся обученные модели морфологического и синтаксического анализа [slovnet](https://github.com/natasha/slovnet), а также русскоязычные тексты из базы [corus](https://natasha.github.io/corus/). Для обучения пригодится заранее размеченная база [nerus](https://natasha.github.io/nerus/).

Приблизительное описание категорий, которые мы можем получить из анализаторов, можно почитать [здесь](https://universaldependencies.org/ru/index.html).

# Установка, импорт и базовая настройка необходимых модулей

In [ ]:
! wget https://storage.yandexcloud.net/natasha-navec/packs/navec_news_v1_1B_250K_300d_100q.tar      # Pretrained word embeddings model weights
! wget https://storage.yandexcloud.net/natasha-slovnet/packs/slovnet_ner_news_v1.tar                # Pretrained Named Entity Recognition model weights
! wget https://storage.yandexcloud.net/natasha-slovnet/packs/slovnet_morph_news_v1.tar              # Pretrained morphological analyzator model weights 
! wget https://storage.yandexcloud.net/natasha-slovnet/packs/slovnet_syntax_news_v1.tar             # Pretrained syntax analyzator model weights

! wget https://github.com/yutkin/Lenta.Ru-News-Dataset/releases/download/v1.0/lenta-ru-news.csv.gz  # Dataset
! wget https://storage.yandexcloud.net/natasha-nerus/data/nerus_lenta.conllu.gz                     # Training dataset

! pip install slovnet
! pip install ipymarkup
! pip install corus
! pip install nerus
! pip install catboost

--2021-04-09 16:29:55--  https://storage.yandexcloud.net/natasha-navec/packs/navec_news_v1_1B_250K_300d_100q.tar
Resolving storage.yandexcloud.net (storage.yandexcloud.net)... 213.180.193.243, 2a02:6b8::1d9
Connecting to storage.yandexcloud.net (storage.yandexcloud.net)|213.180.193.243|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26634240 (25M) [application/x-tar]
Saving to: ‘navec_news_v1_1B_250K_300d_100q.tar’

navec_news_v1_1B_25 100%[===================>]  25.40M  7.94MB/s    in 4.5s    

2021-04-09 16:30:01 (5.60 MB/s) - ‘navec_news_v1_1B_250K_300d_100q.tar’ saved [26634240/26634240]

--2021-04-09 16:30:01--  https://storage.yandexcloud.net/natasha-slovnet/packs/slovnet_ner_news_v1.tar
Resolving storage.yandexcloud.net (storage.yandexcloud.net)... 213.180.193.243, 2a02:6b8::1d9
Connecting to storage.yandexcloud.net (storage.yandexcloud.net)|213.180.193.243|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2385920 (2.3M) [applica

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import make_pipeline as make_pipeline_with_sampler
from ipymarkup import show_dep_ascii_markup as show_markup
from navec import Navec
from slovnet import Syntax, Morph
from razdel import sentenize, tokenize
import re
from corus import load_lenta
from nerus import load_nerus
from functools import reduce
from catboost import CatBoostClassifier, Pool, cv

In [ ]:
navec = Navec.load('navec_news_v1_1B_250K_300d_100q.tar')

syntax = Syntax.load('slovnet_syntax_news_v1.tar')
syntax.navec(navec)

morph = Morph.load('slovnet_morph_news_v1.tar', batch_size=4)
_ = morph.navec(navec)

# Анализ данных и инструментов



Для того, чтобы разобраться, как работать с инструментами, а также для того, чтобы сформировать интуитивное понимание и идею работы будущей модели, нужно протестировать инструменты на примерах.

Посмотрим на синтаксический и морфологический анализ предложения ниже, и попытаемся выявить различия между глаголами "тся" и "ться", а также отфильтруем лишнюю информацию.

Сначала разобъем предложение на токены.

In [ ]:
def create_chunks(text):
  chunk = []
  for sent in sentenize(text):
      tokens = [token.text for token in tokenize(sent.text)]
      chunk.append(tokens)
  return chunk

text_with_infinitive = 'Моя модель сможет обучиться?'

text_without_infinitive = 'Моя модель обучится?'

infinitive_chunks = create_chunks(text_with_infinitive)

non_infinitive_chunks = create_chunks(text_without_infinitive)

print(infinitive_chunks)
print()
print(non_infinitive_chunks)

[['Моя', 'модель', 'сможет', 'обучиться', '?']]

[['Моя', 'модель', 'обучится', '?']]


Теперь посмотрим, какую морфологическую информацию по нашему предложению нам выдаёт модель.

In [ ]:
def print_morph(chunks):
  markup = next(morph.map(chunks))
  for token in markup.tokens:
      print(f'{token.text:>20} {token.tag}')

print_morph(infinitive_chunks)
print()
print_morph(non_infinitive_chunks)

MorphToken(text='Моя', pos='DET', feats={'Case': 'Nom', 'Gender': 'Fem', 'Number': 'Sing'})
                 Моя DET|Case=Nom|Gender=Fem|Number=Sing
MorphToken(text='модель', pos='NOUN', feats={'Animacy': 'Inan', 'Case': 'Nom', 'Gender': 'Fem', 'Number': 'Sing'})
              модель NOUN|Animacy=Inan|Case=Nom|Gender=Fem|Number=Sing
MorphToken(text='сможет', pos='VERB', feats={'Aspect': 'Perf', 'Mood': 'Ind', 'Number': 'Sing', 'Person': '3', 'Tense': 'Fut', 'VerbForm': 'Fin', 'Voice': 'Act'})
              сможет VERB|Aspect=Perf|Mood=Ind|Number=Sing|Person=3|Tense=Fut|VerbForm=Fin|Voice=Act
MorphToken(text='обучиться', pos='VERB', feats={'Aspect': 'Perf', 'VerbForm': 'Inf', 'Voice': 'Mid'})
           обучиться VERB|Aspect=Perf|VerbForm=Inf|Voice=Mid
MorphToken(text='?', pos='PUNCT', feats={})
                   ? PUNCT

MorphToken(text='Моя', pos='DET', feats={'Case': 'Nom', 'Gender': 'Fem', 'Number': 'Sing'})
                 Моя DET|Case=Nom|Gender=Fem|Number=Sing
MorphToken(text='

Помимо частей речи и пунктуационных знаков, мы также получаем развёрнутую информацию о части речи, например, её число, пол.

Посмотрим повнимательнее на интересующие нас глаголы: "Обучиться" и "Обучится".

Видим, что среди прочего нам доступен параметр VerbForm. Именно его нам и нужно научиться предсказывать.

Ясно, что форма глагола зависит в большой степени от слов, которые находятся рядом с глаголом. Поэтому в качестве baseline признаков мы возьмём части речи соседей интересующего нас глагола.

Кроме того, ясно, что пол (женский, мужской, средний) на форму глагола влиять не должен. (Пример как раз демонстрирует это). Поэтому такая информация нам не нужна. По тем же соображениям можно отфильтровать информацию о времени, числе, и залоге.

Ещё одна причина к фильтрации этой информации заключается в том, что единственный общий параметр, который есть у всех слов -- это часть речи. Поэтому начать с неё проще всего.


Ещё важно заметить, что необязательно ограничиваться лишь на словах, оканчивающихся на "тся" и "ться". Мы можем использовать любой глагол для тренировки модели, и обобщить задачу до "классификация личной или инфинитивной формы глагола".

Тогда, если глагол в инфинитиве, то на конце должно быть "ться". Если в личной форме, то "тся".

Перейдём к синтаксическому анализу.

In [ ]:
def print_syntax(chunks):
    markup = next(syntax.map(chunks))

    words, deps = [], []
    for token in markup.tokens:
        words.append(token.text)
        source = int(token.head_id) - 1
        target = int(token.id) - 1
        if source > 0 and source != target:  # skip root, loops
            deps.append([source, target, token.rel])
    show_markup(words, deps)

print_syntax(infinitive_chunks)
print()
print_syntax(non_infinitive_chunks)

    ┌► Моя       det
  ┌►└─ модель    nsubj
┌─└─┌─ сможет    
│   └► обучиться xcomp
└────► ?         punct

  ┌► Моя      det
┌►└─ модель   nsubj
└─┌─ обучится 
  └► ?        punct


Здесь мы видим, как слова связаны друг с другом. Это даёт нам какую-то информацию о посыле предложения, о том, зачем использовано то или иное слово.

Мы можем использовать эту информацию в признаках. 

Чтобы это сделать, возьмём "указатель" на интересующее слово (он всегда один, поскольку слово является узлом в дереве), и несколько "указателей", которые идут из текущего слова. 

Таким образом, мы получим "нелинейные" координаты, и сможем заиспользовать только те слова, которые непосредственно имеют отношение к классифицируемому объекту.

Например, в предложении "Я знаю, что у нас, невзирая на все трудности, получится." все "линейные" соседи слова - это придаточная часть, которая не имеет прямого отношения к глаголу, и, соответственно, классификация на этом предложении сломается.

Используя такие древовидные координаты, мы избегаем описанной проблемы.
Помимо непосредственно части речи в древовидных координатах, мы можем добавить и информацию о типе связи.

Например, слово "обучиться" в примере выше является [xcomp](https://universaldependencies.org/u/dep/xcomp.html) по отношению к слову "сможет". Лишней эта информация не будет.

_____
Теперь предлагаю посмотреть на поведение анализаторов в предложении, содержащем ошибку.

In [ ]:
mistake_chunks = create_chunks('Моя модель сможет обучится?')

print_morph(mistake_chunks)
print()
print_syntax(mistake_chunks)

                 Моя DET|Case=Nom|Gender=Fem|Number=Sing
              модель NOUN|Animacy=Inan|Case=Nom|Gender=Fem|Number=Sing
              сможет VERB|Aspect=Perf|Mood=Ind|Number=Sing|Person=3|Tense=Fut|VerbForm=Fin|Voice=Act
            обучится VERB|Aspect=Imp|VerbForm=Inf|Voice=Act
                   ? PUNCT

    ┌► Моя      det
  ┌►└─ модель   nsubj
┌─└─┌─ сможет   
│   └► обучится xcomp
└────► ?        punct


Возможно, нам повезло, но синтаксическое дерево не изменилось совсем, а в морфологическом анализе даже правильно (с точки зрения смысла, а не реальной формы глагола) была предсказана форма глагола. В связи с этим, добавим к признакам также и предсказанный VerbForm.


Посмотрим на 3 случайных предложения, содержащих интересующие нас глаголы.


In [ ]:
verb_regex = re.compile('.*(тся|ться)+.*')

records = load_lenta('lenta-ru-news.csv.gz')

counter = 0

while counter < 3:
  for sent in sentenize(next(records).text):
    if verb_regex.match(sent.text):
      counter += 1
      print_morph(create_chunks(sent.text))
      print()
      print_syntax(create_chunks(sent.text))
      print()

         Биатлонисту PROPN|Animacy=Anim|Case=Dat|Gender=Masc|Number=Sing
              Антону PROPN|Animacy=Anim|Case=Dat|Gender=Masc|Number=Sing
            Шипулину PROPN|Animacy=Anim|Case=Dat|Gender=Masc|Number=Sing
                   , PUNCT
               также ADV|Degree=Pos
           попавшему VERB|Aspect=Perf|Case=Dat|Gender=Masc|Number=Sing|Tense=Past|VerbForm=Part|Voice=Act
                   в ADP
              список NOUN|Animacy=Inan|Case=Acc|Gender=Masc|Number=Sing
                   , PUNCT
             полиция NOUN|Animacy=Inan|Case=Nom|Gender=Fem|Number=Sing
             нанесла VERB|Aspect=Perf|Gender=Fem|Mood=Ind|Number=Sing|Tense=Past|VerbForm=Fin|Voice=Act
           отдельный ADJ|Animacy=Inan|Case=Acc|Degree=Pos|Gender=Masc|Number=Sing
               визит NOUN|Animacy=Inan|Case=Acc|Gender=Masc|Number=Sing
                   : PUNCT
              сейчас ADV|Degree=Pos
                  он PRON|Case=Nom|Gender=Masc|Number=Sing|Person=3
         тренируется VERB|As

Заметим ещё одну особенность задачи: в представленной случайной выборке количество предложений, содержащих инфинитив - 0.

Это говорит нам о том, что в реальной жизни люди чаще используют личную форму глагола (что, в общем то, верно). 

Возможно, придётся обучить дополнительную модель на сбалансированных данных, и добавить ответ этой модели к признакам другой модели, обученной целиком.

Однако, пока предлагаю об этом не думать.

На этом анализ данных завершён, перейдём к препроцессингу.

# Препроцессинг


Для обучения будем использовать размеченные данные из [nerus](https://natasha.github.io/nerus/).

Из них мы будем извлекать target-variable. Но, несмотря на то, что в данных размечены и части речи, и взаимоотношения между словами, всё равно необходимо сделать извлечь эти данные заново с помощью моделей, описанных выше, потому что опираться мы должны на качество этих моделей, а не на качество разметки.

Кроме того, в списке признаков присутствует морфологическое предсказание модели, что является target-variable, что не подходит для обучения.

Как мы выяснили раньше, нам нужно подготовить для каждого слова в инфинитивной/личной форме следующие признаки:
- части речи из k-окрестности слова 
- VerbForm, предсказанную морфологическим анализатором
- родитель в узле синтаксического дерева
- тип связи с родителем в синтаксическом дереве
- m детей в синтаксическом дереве
- типы связей с детьми в синтаксическом дереве
- (опционально) предсказание модели, обученной на сбалансированной выборке на признаках, описанных выше


Посмотрим на сам датасет.

In [ ]:
docs = load_nerus('nerus_lenta.conllu.gz')

next(docs)

NerusDoc(
    id='0',
    sents=[NerusSent(
         id='0_0',
         text='Вице-премьер по социальным вопросам Татьяна Голикова рассказала, в каких регионах России зафиксирована наиболее высокая смертность от рака, сообщает РИА Новости.',
         tokens=[NerusToken(
              id='1',
              text='Вице-премьер',
              pos='NOUN',
              feats={'Animacy': 'Anim',
               'Case': 'Nom',
               'Gender': 'Masc',
               'Number': 'Sing'},
              head_id='7',
              rel='nsubj',
              tag='O'
          ),
          NerusToken(
              id='2',
              text='по',
              pos='ADP',
              feats={},
              head_id='4',
              rel='case',
              tag='O'
          ),
          NerusToken(
              id='3',
              text='социальным',
              pos='ADJ',
              feats={'Case': 'Dat', 'Degree': 'Pos', 'Number': 'Plur'},
              head_id='4',
             

Каждый документ содержит набор предложений. Каждое предложение содержит набор слов. У каждого слова указан его родитель, отношение к родителю, морфологическая информация, и часть речи.

In [ ]:
class FeatureNames:
  MorphPrediction = 'morph_prediction'
  ParentPos = 'parent_pos'
  ParentRel = 'parent_rel'

  def NextIthPos(i):
    return 'next_{0}th_pos'.format(i)
  
  def PrevIthPos(i):
    return 'prev_{0}th_pos'.format(i)
  
  def IthChildPos(i):
    return '{0}_child_pos'.format(i)
  
  def IthChildRel(i):
    return '{0}_child_rel'.format(i)

def create_columns_for_X(k, m):
  X_columns = [FeatureNames.MorphPrediction, FeatureNames.ParentPos, FeatureNames.ParentRel]

  for i in range(k):
    X_columns.append(FeatureNames.NextIthPos(i + 1))
    X_columns.append(FeatureNames.PrevIthPos(i + 1))
  
  for i in range(m):
    X_columns.append(FeatureNames.IthChildPos(i + 1))
    X_columns.append(FeatureNames.IthChildRel(i + 1))

  return X_columns

class SyntaxToken:
  def __init__(self, parent_sentence_id, parent_word_id, relation):
    self.parent_sentence_id = parent_sentence_id
    self.parent_word_id = parent_word_id
    self.relation = relation

class MorphToken:
  def __init__(self, pos, feats):
    self.pos = pos
    self.feats = feats

class FeatureToken:
  def __init__(self, sentence_id, word_id, syntax_token, morph_token, word):
    self.sentence_id = sentence_id
    self.word_id = word_id
    self.syntax_token = syntax_token
    self.morph_token = morph_token
    self.word = word

def parse_text_into_feature_tokens(text):
  chunks = create_chunks(text)
  morph_chunks = []
  syntax_chunks = []
  feature_tokens = []

  for i, (syntax_res, morph_res) in enumerate(zip(syntax.map(chunks), morph.map(chunks))):
    morph_chunks.append([])
    syntax_chunks.append([])
    for j, (syntax_token, morph_token) in enumerate(zip(syntax_res.tokens, morph_res.tokens)):
      parent = int(syntax_token.head_id) - 1
      syntax_chunks[-1].append(SyntaxToken(i, parent, syntax_token.rel))
      morph_chunks[-1].append(MorphToken(morph_token.pos, morph_token.feats))
      feature_tokens.append(FeatureToken(i, j, syntax_chunks[-1][-1], morph_chunks[-1][-1], syntax_token.text))
  
  return feature_tokens, morph_chunks, syntax_chunks

def fill_features(sequence_index, token, k, m, feature_tokens, morph_chunks, syntax_chunks):
  features = {}
  features[FeatureNames.MorphPrediction] = token.morph_token.feats['VerbForm'] if 'VerbForm' in token.morph_token.feats else ""
  features[FeatureNames.ParentPos] = morph_chunks[token.syntax_token.parent_sentence_id][token.syntax_token.parent_word_id].pos
  features[FeatureNames.ParentRel] = token.syntax_token.relation

  for i in range(1, k + 1):
    next_ith_token = feature_tokens[sequence_index + i] if sequence_index + i < len(feature_tokens) else None
    prev_ith_token = feature_tokens[sequence_index - i] if sequence_index - i >= 0 else None

    features[FeatureNames.NextIthPos(i)] = morph_chunks[next_ith_token.sentence_id][next_ith_token.word_id].pos if next_ith_token else ""
    features[FeatureNames.PrevIthPos(i)] = morph_chunks[prev_ith_token.sentence_id][prev_ith_token.word_id].pos if prev_ith_token else ""
  
  for i in range(1, m + 1):
    features[FeatureNames.IthChildPos(i)] = ""
    features[FeatureNames.IthChildRel(i)] = ""
  
  child_counter = 0

  for i, word in enumerate(syntax_chunks[token.sentence_id]):
    if child_counter == m:
      break
    
    if word.parent_word_id == token.word_id:
      child_counter += 1
      features[FeatureNames.IthChildPos(child_counter)] = morph_chunks[token.sentence_id][i].pos
      features[FeatureNames.IthChildRel(child_counter)] = word.relation
    
  return features

def extract_features(text, k, m, selector):
  feature_tokens, morph_chunks, syntax_chunks = parse_text_into_feature_tokens(text)

  for i, token in enumerate(feature_tokens):
    if selector(token):
      yield fill_features(i, token, k, m, feature_tokens, morph_chunks, syntax_chunks), token


def concat_words(prev, curr):
  return prev + " " + curr


def get_text_from_nerus_doc(doc):
  return reduce(concat_words, map(lambda x: x.text, doc.sents))

def generate_rows_by_doc(doc, k, m):
  selector = lambda x: x.morph_token.pos == 'VERB'

  for features, token in extract_features(get_text_from_nerus_doc(doc), k, m, selector):
    if (doc.sents[token.sentence_id].tokens[token.word_id].pos == 'VERB' and
     'VerbForm' in doc.sents[token.sentence_id].tokens[token.word_id].feats and
      (doc.sents[token.sentence_id].tokens[token.word_id].feats['VerbForm'] == "Inf" or
       doc.sents[token.sentence_id].tokens[token.word_id].feats['VerbForm'] == "Fin")):
      yield features, {'VerbForm': doc.sents[token.sentence_id].tokens[token.word_id].feats['VerbForm']}


def log_progress(sequence, every=10):
    from ipywidgets import IntProgress
    from IPython.display import display

    progress = IntProgress(min=0, max=len(sequence), value=0)
    display(progress)
    
    for index, record in enumerate(sequence):
        if index % every == 0:
            progress.value = index
        yield record

def create_test_dataframes(doc_amount, k, m):
  X_data = []
  Y_data = []

  docs = load_nerus('nerus_lenta.conllu.gz')

  for i in log_progress(range(doc_amount)):
    doc = next(docs, None)
    if not doc:
      print("Very big amount of docs requested! Returning with {0} docs.".format(i))
      break
    
    for X_row, Y_val in generate_rows_by_doc(doc, k, m):
      X_data.append(X_row)
      Y_data.append(Y_val)
  
  return pd.DataFrame(data=X_data, columns=create_columns_for_X(k, m)), pd.DataFrame(data=Y_data, columns=['VerbForm'])


def generate_dataframe_by_text(text, k, m, selector):
  features = []

  for feature, _ in extract_features(text, k, m, selector):
    features.append(feature)
  
  return pd.DataFrame(data=features, columns=create_columns_for_X(k, m))


Теперь мы умеем генерировать датасеты. Посмотрим на полученные признаки.

In [ ]:
X_test, Y_test = create_test_dataframes(1, 2, 1)    

print(X_test)
print()
print(Y_test)

IntProgress(value=0, max=1)

  morph_prediction parent_pos parent_rel  ... prev_2th_pos 1_child_pos 1_child_rel
0              Fin      PUNCT       root  ...        PROPN        NOUN       nsubj
1              Fin      PUNCT       root  ...         NOUN       PUNCT       punct
2              Fin      PUNCT       root  ...          ADJ        NOUN   parataxis
3              Fin      PUNCT       root  ...        PUNCT       PROPN       nsubj
4              Fin      PUNCT       root  ...         NOUN        NOUN         obl
5              Fin       VERB      ccomp  ...          ADP       PUNCT       punct
6              Fin      PUNCT       root  ...          ADP        NOUN   parataxis

[7 rows x 9 columns]

  VerbForm
0      Fin
1      Fin
2      Fin
3      Fin
4      Fin
5      Fin
6      Fin


Похоже, что всё корректно обработалось.

Запустим, чтобы создать первоначальную выборку.

In [ ]:
%%time
X_test, Y_test = create_test_dataframes(10000, 3, 3)   

IntProgress(value=0, max=10000)

CPU times: user 12min 54s, sys: 7min 21s, total: 20min 15s
Wall time: 10min 28s


Сохраним данные.

In [ ]:
X_test.to_csv('X_test.csv', index=False)
Y_test.to_csv('Y_test.csv', index=False)

Теперь посмотрим на соотношение инфинитивов и личной формы глагола

In [ ]:
Y_test.value_counts()

VerbForm
Fin         170706
Inf          32973
dtype: int64

Дела не так уж и плохи, примерно 15% выборки содержит меньший класс, что довольно неплохо.

# Обучение модели

Все наши признаки являются категориальными.

Так как мне лень подбирать кодирование, для начала я попробую применить [CatBoost](https://catboost.ai/docs/concepts/about.html), который из коробки умеет работать с категориальными фичами.

Загрузим подготовленный выше датасет

In [ ]:
! wget https://github.com/HolyPrapor/InfinitiveOrFiniteClassification/raw/main/X_test.csv
! wget https://github.com/HolyPrapor/InfinitiveOrFiniteClassification/raw/main/Y_test.csv

In [ ]:
X_test = pd.read_csv('X_test.csv')
Y_test = pd.read_csv('Y_test.csv').replace("Inf", 0).replace("Fin", 1)

In [ ]:
Y_test.value_counts()

VerbForm
1           170706
0            32973
dtype: int64

In [ ]:
cv_dataset = Pool(data=X_test,
                  label=Y_test,
                  cat_features=list(range(len(create_columns_for_X(3, 3)))))

params = {"iterations": 100,
          "depth": 8,
          "loss_function": "Logloss",
          "verbose": True}

cv(cv_dataset,
            params,
            fold_count=2, 
            plot="True")

In [ ]:
model = CatBoostClassifier(iterations=100,
                           depth=8,
                           learning_rate=1,
                           loss_function='Logloss',
                           verbose=True)


model.fit(train_x, train_y, list(range(len(create_columns_for_X(3, 3)))))


In [ ]:
model.feature_importances_

array([ 6.83699936, 22.39200103,  7.02790941, 13.1944267 , 12.81376336,
        3.7745402 ,  4.437716  ,  4.68766638,  2.55328642,  0.        ,
        4.57494639,  7.45417691,  0.        ,  4.94655372,  5.30601412])

In [ ]:
test_text = "Заниматься"


model.predict(generate_dataframe_by_text(test_text, 3, 3, lambda x: x.word.endswith("тся") or x.word.endswith("ться")))

array([0])